In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [9]:
try:
    conn = psycopg2.connect(dbname='mimic', user='tianranzhang')
except:
        print("I am unable to connect to the database")

cur = conn.cursor()
cur.execute("""set search_path to mimiciii""")
cur.execute("""SELECT subject_id, charttime, event_type, event, icd9_3, hadm_id
            from allevents order by subject_id, charttime, case event_type when 'condition' then 1
            when 'symptom' then 2 when 'labevent' then 3 when 'diagnosis' then 4
            when 'prescription' then 5 end, event""")
rows = cur.fetchall()
print("Query executed")

Query executed


In [12]:
pd.DataFrame(rows).to_csv('allevents_info.csv',index=False)

In [14]:
events = pd.DataFrame(rows)
events.columns = ['subject_id', 'charttime', 'event_type', 'event',' icd9_3', 'hadm_id' ]


,subject_id,charttime,event_type,event,icd9_3,hadm_id
0,17,2134-12-31 16:05:00,condition,V1259,None,194023
1,17,2134-12-31 16:05:00,diagnosis,2724,272,194023
2,17,2134-12-31 16:05:00,diagnosis,45829,458,194023
3,17,2134-12-31 16:05:00,diagnosis,7455,745,194023
4,17,2135-05-13 14:40:00,symptom,78551,None,161087
5,17,2135-05-13 14:40:00,diagnosis,2724,272,161087
6,17,2135-05-13 14:40:00,diagnosis,311,311,161087
7,17,2135-05-13 14:40:00,diagnosis,4239,423,161087
8,17,2135-05-13 14:40:00,diagnosis,4589,458,161087
9,17,2135-05-13 14:40:00,diagnosis,5119,511,161087


In [36]:
try:
    conn = psycopg2.connect(dbname='mimic', user='tianranzhang')
except:
        print("I am unable to connect to the database")

cur = conn.cursor()
cur.execute("""set search_path to mimiciii""")
cur.execute("""select FLOOR(hadm_id), subject_id, charttime, 
'labevent' as event_type, itemid as event, flag as event_info from labevents
where  itemid in
(
  select itemid from
  (
    select hadm_id, itemid from labevents where flag='abnormal' group by hadm_id, itemid
  ) as uniqlab
  group by itemid having count(*) > 50
)
and subject_id in (select subject_id from admissions group by subject_id having count(*) > 1)
and hadm_id is not NULL
order by subject_id, charttime
""")
labs = cur.fetchall()
print("Query executed")

Query executed


In [8]:
try:
    conn = psycopg2.connect(dbname='mimic', user='tianranzhang')
except:
        print("I am unable to connect to the database")

cur = conn.cursor()
cur.execute("""set search_path to mimiciii""")
cur.execute("""select FLOOR(a.hadm_id), a.subject_id, a.charttime, 
'labevent' as event_type, b.loinc_code as event, a.flag as event_info 
from labevents a, d_labitems b
where a.itemid = b.itemid
and a.itemid in
(
  select itemid from
  (
    select hadm_id, itemid from labevents where flag='abnormal' group by hadm_id, itemid
  ) as uniqlab
  group by itemid having count(*) > 50
)
and a.subject_id in (select subject_id from admissions group by subject_id having count(*) > 1)
and a.hadm_id is not NULL
order by a.subject_id, a.charttime
""")
labs = cur.fetchall()
print("Query executed")

Query executed


In [12]:
labs = pd.DataFrame(labs)
labs.columns = ['hadm_id','subject_id', 'charttime', 'event_type','event','event_info']
labs.count()

hadm_id       7580984
subject_id    7580984
charttime     7580984
event_type    7580984
event         7571755
event_info    3370295
dtype: int64

In [13]:
labs.event.value_counts()
#189 unique lab codes with mapped loinc_id

4544-3     281745
2823-3     277072
2951-2     265588
718-7      263544
2075-0     261970
2160-0     260251
3094-0     259400
1963-8     257565
1863-0     254448
2345-7     253860
777-3      245330
804-5      237276
2601-3     236950
786-4      235589
789-8      235529
785-6      235529
787-2      235528
788-0      235221
2777-1     213162
2000-8     211073
11558-4    182743
34728-6    169624
11556-8    169609
11557-6    169598
3173-2     169258
5895-7     158790
5902-2     158703
1994-3      84300
32693-4     69102
1975-2      66768
            ...  
28544-5       194
26469-7       191
26522-3       191
2857-1        180
26458-0       173
6928-6        165
18262-6       150
3209-4        148
32215-6       144
13047-6       134
11153-4       117
13965-9        95
766-6          95
5820-6         90
27818-4        80
2842-3         67
27811-9        66
32546-4        64
33369-0        64
33370-8        61
4049-3         60
9842-6         57
10334-1        52
33558-8        41
26510-8   

In [14]:
labs.to_csv('labevents_info.csv',index=False)

In [ ]:
try:
    conn = psycopg2.connect(dbname='mimic', user='tianranzhang')
except:
        print("I am unable to connect to the database")

cur = conn.cursor()
cur.execute("""set search_path to mimiciii""")
cur.execute("""select hadm_id, subject_id, starttime as charttime, 'prescription' as event_type, 
formulary_drug_cd as event 
from prescriptions
where formulary_drug_cd in
(
  select formulary_drug_cd from
  (
    select hadm_id, formulary_drug_cd from prescriptions group by hadm_id, formulary_drug_cd
  ) as uniqlab
  group by formulary_drug_cd having count(*) > 50
)
and subject_id in (select subject_id from admissions group by subject_id having count(*) > 1)
""")
meds = cur.fetchall()
print("Query executed")